## 0. Import libraries

In [1]:
import pandas as pd
import os
from googleapiclient.discovery import build
from typing import List
#pip install google-api-python-client

## 1. Get API key and connect to Youtube API

In [2]:
yt_api_key = os.environ.get('YT_API_KEY')
#print(yt_api_key)
yt_connection = build('youtube', 'v3', developerKey = yt_api_key)

**a) Test**

In [4]:
request = yt_connection.channels().list(
    part = 'statistics',
    forUsername='schafer5' # Corey Schafter
    )
response = request.execute()

In [5]:
response

{'kind': 'youtube#channelListResponse',
 'etag': 'eTvhligLCojh9BDKXsDlHEQOoEI',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'x_bxx3k2_8eK8b6rl2JGOVypYLU',
   'id': 'UCCezIgC97PvUuR4_gbFUs5g',
   'statistics': {'viewCount': '90022941',
    'subscriberCount': '1220000',
    'hiddenSubscriberCount': False,
    'videoCount': '232'}}]}

## 2. Create a function to extract comments for given youtube video

**How to insert youtube video id?**<br>
https://www.youtube.com/watch?v= **dQw4w9WgXcQ**<br>
The bolded text functions as a youtube video ID.

In [8]:
def video_comments(video_id: str) -> List:
    # empty list for storing replies
    replies = []
    all_replies = []
    # creating youtube resource object
    yt_connection = build('youtube', 'v3', developerKey = yt_api_key)
 
    # retrieve youtube video results
    video_response=yt_connection.commentThreads().list(
        part='snippet,replies',
        videoId=video_id,
        maxResults = 100 # MAX 100
        ).execute()
    # iterate video response
    while video_response:
        # extracting required info
        # from each result object
        for item in video_response['items']:
            # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            all_replies.append(comment)
            # counting number of reply of comment
            replycount = item['snippet']['totalReplyCount']
 
            # if reply is there
            if replycount>0:
               
                # iterate through all reply
                for reply in item['replies']['comments']:
                   
                    # Extract reply
                    reply = reply['snippet']['textDisplay']
                    
                    # Store reply is list
                    replies.append(reply)
                    all_replies.append(reply)
 
            # empty reply list
            replies = []
        if len(all_replies) > 5000:
            break
         #Again repeat
        if 'nextPageToken' in video_response:
            video_response = yt_connection.commentThreads().list(
                    part = 'snippet,replies',
                    videoId = video_id,
                    maxResults = 100,
                    pageToken=video_response['nextPageToken']
                    ).execute()
        else:
            break
    return all_replies

**Function to extract video_id from a youtube link**

In [9]:
from urllib.parse import urlparse, parse_qs

# source:
# https://stackoverflow.com/questions/45579306/get-youtube-video-url-or-youtube-video-id-from-a-string-using-regex
def get_id(url):
    u_pars = urlparse(url)
    quer_v = parse_qs(u_pars.query).get('v')
    if quer_v:
        return quer_v[0].strip()
    pth = u_pars.path.split('/')
    if pth:
        return pth[-1].strip()

In [10]:
id_url = 'https://www.youtube.com/watch?v=m_otDNRqK1M&t=4s'
video_id = get_id(id_url)
video_id

'm_otDNRqK1M'

In [11]:
# Call function
result = video_comments(video_id)

In [12]:
len(result)

2519

The other value of interest for the commentThreads resource was the nextPageToken. Each time we submit a request, we get maxResults number of comments in the items list. The maximum number of results we can obtain is limited between 1 and 100. Thus, if a video has more than 100 comments, we’ll need to make an API call several times. The nextPageToken helps us start directly on the next page of comments instead of starting from the beginning again. We just need to modify our API call a bit:

**Get the title**

In [13]:
# response to get the title of the video
response_title = yt_connection.videos().list(
     part = 'snippet',
     id = video_id
).execute()
# get the video title
video_title = response_title['items'][0]['snippet']['title']
video_title

'Dlaczego Rosja to najgorszy kraj na świecie?'

In [15]:
from datetime import date

today = str(date.today())[5:].replace('-','_')
print("Today's date:", today)

Today's date: 10_29


In [16]:
# dictionary of lists
comms = {f'Comments- video title:{video_title}, link:{id_url}':result}
     
df = pd.DataFrame(comms)

In [17]:
df

,"Comments- video title:Dlaczego Rosja to najgorszy kraj na świecie?, link:https://www.youtube.com/watch?v=m_otDNRqK1M&t=4s"
0,Który argument najbardziej Cię zaskoczył? 🤔🤔🤔
1,To jest Rosja. Tam nic nie zaskakuje.
2,HIV
3,Jesteś ulepiony z kału aborygenów wierzących w...
4,Ciebie na gnojowniku znaleźli
...,...
2514,Pierwszy
2515,Godamn prawie bym zdążył
2516,Debil
2517,dumny z siebie jesteś?


### Save comments to file

In [ ]:
path = r'C:\Users\Chill\Desktop\INZYNIERKA\dane\extracted_comments\comms{}.xlsx'.format(today)
# saving the dataframe
df.to_excel(path, index=False, header=True,encoding="utf-8")

## Import libraries

In [1]:
import os
from googleapiclient.discovery import build
from typing import List
#pip install google-api-python-client

## Insert youtube video ID <br>
**How to insert youtube video id?**<br>
https://www.youtube.com/watch?v= **dQw4w9WgXcQ**<br>
The bolded text functions as a youtube video ID.

In [2]:
# Enter video id
video_id = "hyRhzXRAF_I"

**Function to extract video_id from a youtube link**

In [3]:
from urllib.parse import urlparse, parse_qs

# source:
# https://stackoverflow.com/questions/45579306/get-youtube-video-url-or-youtube-video-id-from-a-string-using-regex
def get_id(url):
    u_pars = urlparse(url)
    quer_v = parse_qs(u_pars.query).get('v')
    if quer_v:
        return quer_v[0].strip()
    pth = u_pars.path.split('/')
    if pth:
        return pth[-1].strip()

In [4]:
video_id = get_id('https://www.youtube.com/watch?v= dQw4w9WgXcQ')
video_id

'dQw4w9WgXcQ'

## Get API key and connect to Youtube API

In [5]:
yt_api_key = os.environ.get('YT_API_KEY')
#print(yt_api_key)
yt_connection = build('youtube', 'v3', developerKey = yt_api_key)

In [6]:
# Enter video id
video_id = "hyRhzXRAF_I"

### - Get the title of a video -

In [7]:
# response to get the title of the video
response_title = yt_connection.videos().list(
     part = 'snippet',
     id = video_id
).execute()
# get the video title
video_title = response_title['items'][0]['snippet']['title']
video_title

'CLUB2020 NA OPEN’ERZE!'